### Importing essential libraries 

In [1]:
import pandas as pd 
import numpy as np 
import requests 
from bs4 import BeautifulSoup


link = 'https://github.com/AshishJangra27'

soup = BeautifulSoup(requests.get(link).content,'html.parser')

### Defining a social account functions 

In [2]:
def social_account(urls):
    accounts = []
    for url in urls:
        if 'kaggle' in url:
            accounts.append('Kaggle')
        elif 'instagram' in url:
            accounts.append('Instagram')
        elif 'facebook' in url:
            accounts.append('Facebook')
        elif 'linkedin' in url:
            accounts.append('Linkedin')
        elif 'github' in url:
            accounts.append('Github')
        else:
            accounts.append('Personal website')
    return ','.join(accounts)

### Scraping the data of the github repositories 

In [3]:
user_name  = soup.find('span',class_ = 'p-name vcard-fullname d-block overflow-hidden').text.strip()
user_id    = soup.find('span',class_ = 'p-nickname vcard-username d-block').text.strip()
bio        = soup.find('div',class_ = 'js-profile-editable-area d-flex flex-column d-md-block').find('div').text
lives_in   = soup.find('li',class_ = 'vcard-detail pt-1 hide-sm hide-md').text.strip()
data = []

for i in soup.find_all('a',class_ = 'Link--secondary no-underline no-wrap'):
    if 'follower' in (i.text):
        follower = int(i.find('span').text.strip())
    elif 'following' in i.text:
        following = int(i.find('span').text.strip())
    try:        
        content  = [int(content.find('span').text.strip()) for content in soup.find('nav',class_ ='UnderlineNav-body width-full p-responsive js-sidenav-container-pjax').find_all('a')[1:]]
        repos    = content[0]
        project  = content[1]
        packages = content[2]
        stars    = content[3]
    except:
        repos    = np.nan
        project  = np.nan
        packages = np.nan
        stars    = np.nan
    try:
        url = [links.find('a')['href'] for links in soup.find_all('li',class_='vcard-detail pt-1')]
        account = social_account(url)
    except:
        url = np.nan
        account = 'no social account has been linked'


data.append([user_id , user_name , bio , follower , following ,lives_in,url,account, repos , project , packages , stars])

df = pd.DataFrame(data,columns = ['user_id' ,'user_name' ,'bio' ,'follower' ,'following', 'lives_in', 'url','account', 'repos' , 'project' , 'packages' , 'stars'])
df.head()




,user_id,user_name,bio,follower,following,lives_in,url,account,repos,project,packages,stars
0,AshishJangra27,Ashish Jangra,I am an AI Trainer. \nI train students on tech...,303,1,Noida,[],,76,0,0,52


### Scraping the repos

In [8]:
link = 'https://github.com/AshishJangra27?tab=repositories'
repos_links = []
repos_name  = []

for pages in range(int(round(df['repos']/30))):
    res = requests.get(link)
    soup = BeautifulSoup(res.content,'html.parser')
    for name in soup.find_all('div',class_ ='d-inline-block mb-1'):
        repos_links.append('https://www.github.com' + name.find('a').get('href'))
    for name in soup.find_all('div',class_ ='d-inline-block mb-1'):
        repos_name.append(name.find('a').text.strip())
    try:
        link = 'https://www.github.com' + soup.find('div',{'class':'paginate-container'}).find('a',class_ = 'next_page').get('href')    
    except:
        pass

### Scraping the stars

In [7]:
link = 'https://github.com/AshishJangra27?tab=stars'

stars_repo_link = []
stars_repo_name = []

for pages in range(int(round(df['stars']/30))):
    res = requests.get(link).text
    soup = BeautifulSoup(res,'html.parser')
    for stars in soup.find_all('div',class_ = 'd-inline-block mb-1'):
        stars_repo_link.append('https://www.github.com' + stars.find('a').get('href'))
    for stars in soup.find_all('div',class_ = 'd-inline-block mb-1'):
        stars_repo_name.append(stars.find('a').text.strip().split('/')[1].strip())
    try:
        link = soup.find('div',{'class':'paginate-container'}).find('a',class_ = 'next_page').get('href')
    except:
        pass

### Concating the data 


In [14]:
df_ = pd.DataFrame()

df_['repos_name']       = [repos_name]
df_['repos_links']      = [repos_links]
df_['stars_repos']      = [stars_repo_link]
df_['stars_repos_link'] = [stars_repo_name]

df = pd.concat([df,df_],axis = 1)
df

,user_id,user_name,bio,follower,following,lives_in,url,account,repos,project,packages,stars,repos_name,repos_links,stars_repos,stars_repos_link,repos_name,repos_links,stars_repos,stars_repos_link
0,AshishJangra27,Ashish Jangra,I am an AI Trainer. \nI train students on tech...,303,1,Noida,[],,76,0,0,52,"[In-One-Go, Data-Science-Live-GFG, Airbnb-Scra...",[https://www.github.com/AshishJangra27/In-One-...,[https://www.github.com/AshishJangra27/Gaana-D...,"[Gaana-Data-Analysis, GFG-Tableau-Course-Datas...","[In-One-Go, Data-Science-Live-GFG, Airbnb-Scra...",[https://www.github.com/AshishJangra27/In-One-...,[https://www.github.com/AshishJangra27/Gaana-D...,"[Gaana-Data-Analysis, GFG-Tableau-Course-Datas..."


### Scraping the followers

In [17]:
link = 'https://github.com/AshishJangra27?tab=followers'
res = requests.get(link).text
soup = BeautifulSoup(res,'html.parser')
print(soup)


<!DOCTYPE html>

<html data-a11y-animated-images="system" data-color-mode="auto" data-dark-theme="dark" data-light-theme="light" lang="en">
<head>
<meta charset="utf-8"/>
<link href="https://github.githubassets.com" rel="dns-prefetch"/>
<link href="https://avatars.githubusercontent.com" rel="dns-prefetch"/>
<link href="https://github-cloud.s3.amazonaws.com" rel="dns-prefetch"/>
<link href="https://user-images.githubusercontent.com/" rel="dns-prefetch"/>
<link crossorigin="" href="https://github.githubassets.com" rel="preconnect"/>
<link href="https://avatars.githubusercontent.com" rel="preconnect"/>
<link crossorigin="anonymous" href="https://github.githubassets.com/assets/light-fe3f886b577a.css" media="all" rel="stylesheet"><link crossorigin="anonymous" href="https://github.githubassets.com/assets/dark-a1dbeda2886c.css" media="all" rel="stylesheet"><link crossorigin="anonymous" data-color-theme="dark_dimmed" data-href="https://github.githubassets.com/assets/dark_dimmed-1ad5cf51dfeb.c